In [1]:
!pip install openai -q

## 환경 설정
1. openai 사이트에서 API KEY 발급
  
2. google colab - secret tap 에 `OPENAI_API_KEY`를 등록


In [4]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [5]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

In [12]:
response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "안녕 나는 조성렬이야 ~ 요즘 공부가 어려운데 공부방법좀~ "
        }
      ]
    }
  ],
  response_format={
    "type": "text"
  },
  temperature=1,
  max_completion_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

In [13]:
response

ChatCompletion(id='chatcmpl-BlsA8OVEmKZ1N3oXJ4vKaJw2Wh2of', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='안녕하세요, 조성렬님! 공부가 어렵다고 느끼는 건 많은 사람들이 겪는 일이에요. 효과적인 공부 방법 몇 가지를 소개해드릴게요.\n\n1. **목표 설정**: 구체적이고 실현 가능한 목표를 세워보세요. 예를 들어, 하루에 몇 페이지 읽거나, 특정 개념을 이해하는 것을 목표로 할 수 있습니다.\n\n2. **시간 관리**: 공부할 시간을 정해 일정에 맞춰 공부하세요. Pomodoro 기법처럼 25분 동안 집중하고 5분 쉬는 방식을 사용해보는 것도 좋습니다.\n\n3. **적극적인 학습**: 단순히 읽거나 듣는 것을 넘어서 메모를 하거나 친구에게 설명해보세요. 내용을 다른 방식으로 표현해보는 것이 이해에 큰 도움이 됩니다.\n\n4. **복습**: 정기적으로 복습하는 것이 중요해요. 학습한 내용을 일정한 주기로 돌아보며 잊지 않도록 합니다.\n\n5. **환경 조성**: 집중할 수 있는 환경을 조성해보세요. 방해 요소를 최소화하고 조용한 공간에서 공부하는 것이 좋습니다.\n\n6. **건강 관리**: 좋은 식습관과 충분한 수면, 그리고 규칙적인 운동이 공부에 긍정적인 영향을 줄 수 있어요.\n\n7. **스트레스 관리**: 공부가 힘들 때는 잠시 쉬거나, 취미 활동을 하면서 스트레스를 풀어보세요.\n\n어때요? 도움이 될만한 방법이 있을까요? 더 구체적인 공부 방법이나 주제가 필요하면 말씀해 주세요!', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1750749316, model='gpt-4o-mini-2024-07-18', object

In [11]:
# ChatCompletion(choices=[Choice(message(content))])
response.choices[0].message.content

'안녕 한성규! 졸린 이유는 여러 가지가 있을 수 있어. 예를 들어, 밤에 충분한 수면을 못 취했거나, 스트레스를 받고 있거나, 식사 후에 혈당이 급격히 올라갔다 떨어지면서 졸릴 수 있어. 또한, 환경적인 요인이나 단순히 몸이 피곤해서 그럴 수도 있어. 혹시 평소에 잠을 잘 자고 있어?'

## 프롬프팅의 기본구성

https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/

1. Instruction 지시사항
2. Context 문맥정보 (어떤상황인지 상세히 풀어서)
3. Input Data/Example 입력, 예시 데이터
4. Output Indicator 출력지시

## 기사 제목 교정

- 기자들이 송고한 기사에서 제목을 추출하고, 표현을 조정

- 프랑스 AFP 속보 시스템에서 도입되어 사용

In [15]:
title_before = '테이의 FM 개꿀 라디오 방송에 주목해주세요.'
title_before = '졸라 빡센 작업으로 끼니 거르기 일쑤인 노동자들의 애환'

# 역할/페르소나/지시사항
system_message = """
기사들이 송고한 제목에서 맞춤법, 문법, 의미, 어조등에 있어서 교정작업을 수행해 주세요.

- 기사 제목이 명확하고 주제와 잘 맞도록 조정하세요.
- 독자의 관심을 끌 수 있도록 간결하면서도 임팩트 있는 표현을 사용하세요.
- 어조가 지나치게 감정적이거나 부정적인 경우 표현을 완화하거나, 중립적인 어조로 수정하세요.
- 비속어가 포함되어 있는 경우, 비속어를 반드시 제거하고, 의미를 적절히 유지하도록 제목을 교정하세요.

### Steps ###
1. 기사제목을 읽고 주요내용을 이해하세요.
2. 제목이 전달하고자하는 메세지를 명확하게 반영하는지 검토하세요.
3. 맞춤법, 문법, 의미 전달의 정확성등을 점검하고 적절히 수정하세요.
4. 제목이 자연스럽고, 독자에게 매력적으로 다가갈수 있는지 점검하고 매우 간결하게 정리하세요.


### Output Format ###
기사 원래 제목과 교정된 제목을 다음 형식으로 제공하세요.

- 원래 제목: [기사 원래 제목]
- 교정 제목: [교정 기사 제목]

### Examples ###
- 원래 제목: "어제 서울에서 큰 불이 나 수백명이 대피했다."
- 교정 제목: "서울 대형 화재, 수백명 대피"

- 원래 제목: "전기자동차 판매량 급감에 내연차회사들이 즐거워하는 중입니다."
- 교정 제목: "전기차 판매량 급감에 웃는 내연차회사들"

### Extra Instructions ###
- 제목이 너무 길면, 간결하게 줄이되, 핵심 메세지를 잃어버려서는 안됩니다.
- 지역명, 시간 등의 중요한 정보는 명확하게 유지하세요.
- 제목이 특정집단이나 대상에 대해 중립적이지 않을 경우, 그 표현을 완화하세요.

"""

user_message = f"""
다음 기사제목을 교정해주세요.

제목: {title_before}
"""

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": system_message
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": user_message
        }
      ]
    }
  ],
  response_format={
    "type": "text"
  },
  temperature=1,
  max_completion_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)
print(response.choices[0].message.content)

- 원래 제목: 졸라 빡센 작업으로 끼니 거르기 일쑤인 노동자들의 애환
- 교정 제목: 과도한 업무로 끼니를 거르는 노동자들의 고충


## 함수 리팩토링

In [17]:
# 함수 리팩토링

def correct_news_title(title_before, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
    기사들이 송고한 제목에서 맞춤법, 문법, 의미, 어조등에 있어서 교정작업을 수행해 주세요.

    - 기사 제목이 명확하고 주제와 잘 맞도록 조정하세요.
    - 독자의 관심을 끌 수 있도록 간결하면서도 임팩트 있는 표현을 사용하세요.
    - 어조가 지나치게 감정적이거나 부정적인 경우 표현을 완화하거나, 중립적인 어조로 수정하세요.
    - 비속어가 포함되어 있는 경우, 비속어를 반드시 제거하고, 의미를 적절히 유지하도록 제목을 교정하세요.

    ### Steps ###
    1. 기사제목을 읽고 주요내용을 이해하세요.
    2. 제목이 전달하고자하는 메세지를 명확하게 반영하는지 검토하세요.
    3. 맞춤법, 문법, 의미 전달의 정확성등을 점검하고 적절히 수정하세요.
    4. 제목이 자연스럽고, 독자에게 매력적으로 다가갈수 있는지 점검하고 매우 간결하게 정리하세요.


    ### Output Format ###
    기사 원래 제목과 교정된 제목을 다음 형식으로 제공하세요.

    - 원래 제목: [기사 원래 제목]
    - 교정 제목: [교정 기사 제목]

    ### Examples ###
    - 원래 제목: "어제 서울에서 큰 불이 나 수백명이 대피했다."
    - 교정 제목: "서울 대형 화재, 수백명 대피"

    - 원래 제목: "전기자동차 판매량 급감에 내연차회사들이 즐거워하는 중입니다."
    - 교정 제목: "전기차 판매량 급감에 웃는 내연차회사들"

    ### Extra Instructions ###
    - 제목이 너무 길면, 간결하게 줄이되, 핵심 메세지를 잃어버려서는 안됩니다.
    - 지역명, 시간 등의 중요한 정보는 명확하게 유지하세요.
    - 제목이 특정집단이나 대상에 대해 중립적이지 않을 경우, 그 표현을 완화하세요.

    """

    user_message = f"""
    다음 기사제목을 교정해주세요.

    제목: {title_before}
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "text"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content

title_before = '주말 미친 폭우 예상, 모두들 무사하시길~'
output = correct_news_title(title_before)
print(output)

- 원래 제목: 주말 미친 폭우 예상, 모두들 무사하시길~
- 교정 제목: "주말 폭우 예상, 모두 안전을 기원합니다"


## 영문텍스트에 anki단어장 만들기